따릉이로 전처리를 배울 수 있는 좋은 이유
1. 자료가 많다.
2. 결측값이 많지 않다. 
3. 자료에 오류가 상당히 많다.
    * 이용시간 30000분
    * 이용거리 200km
    * 이용거리 0m 인값 300만개..
4. 데이터 오류는 아니나 자잘하게 제거해야하는 데이터가 많다.
    * 실수로 레버를 내리지 않아 반납되지 않은 경우
    * 정비소로 이동한 경우

5. 새로운 데이터를 만들어 보는 경험을 할 수 있다.
    * 분당 이동거리 계산으로 오류를 쉽게 찾기도 했음.
    * 60분 이상 1분당 500m이상 가기가 힘듬. 그러면 30km인데 18kg의 따릉이로 그게 가능할리가 없음


In [26]:
import os
import re
import pandas as pd
import numpy as np
## 대여소가 새로 생기고 없어지는게 정말 많다.
## 9월 이용내역을 12월 대여소 기준으로 만드려고 해도 문제가 생긴다.


files= os.listdir('D:/git_local_repository/data_for_project/seoul_bike/2021')
files

['seoul_bike_2021.01.csv',
 'seoul_bike_2021.02.csv',
 'seoul_bike_2021.03.csv',
 'seoul_bike_2021.04.csv',
 'seoul_bike_2021.05.csv',
 'seoul_bike_2021.06.csv',
 'seoul_bike_2021.07.csv',
 'seoul_bike_2021.08.csv',
 'seoul_bike_2021.09.csv',
 'seoul_bike_2021.10.csv',
 'seoul_bike_2021.11.csv',
 'seoul_bike_2021.12.csv']

In [27]:
# 2분정도 소요
b = []
# for i in range(6,12) :
for i in range(0,6) :
    file = pd.read_csv(f'D:/git_local_repository/data_for_project/seoul_bike/2021/{files[i]}',encoding='CP949')
    b.append(file)

d:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (4,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [28]:
# 2분 정도 소요 
## 한칸씩 밀린 값 복구라고 하지만 제거 (혹시 몰라 밀린 값도 따로 modified total로 모아둠)
modified_total = []
normal_total =[]
for i in range(6) : 
    seoul_bike_df = b[i]
    try : 
        finding_words = seoul_bike_df['반납대여소번호'].astype(str)
        BM = [ num for num, i in enumerate(finding_words) if re.findall('[가-힣+]',i)]
        errors = seoul_bike_df[seoul_bike_df.index.isin(BM)].drop(columns=['대여 대여소명','반납대여소번호','반납대여소명','이용거리'])
        modified = pd.DataFrame(errors.values,columns=['자전거번호','대여일시','대여 대여소번호','반납일시','반납대여소번호','이용시간','이용거리'])
        modified_total.append(modified)
        normal = seoul_bike_df[~seoul_bike_df.index.isin(BM)]
        normal_total.append(normal)
    except : 
        normal_total.append(seoul_bike_df)

In [29]:
### 정리 순서
# 대여소 주소, 대여소 번호 통일
# 자전거 번호 정리
# 대여일시 2021 제거, 날짜, 시간 구분
# 불필요한 column제거 대여거치대, 반납거치대, 반납일시
# 불필요한 row 제거 1.이용거리 기준보고 outlier 제거 2. 대여일시 3월 31일 제거


In [30]:
# 20초 정도 소요
seoul_bike_df = pd.concat(normal_total)
seoul_bike_df_sub = pd.concat(modified_total)

In [31]:
# 2분 정도 소요
# 이용시간 과한 경우 & 거리 0m인 경우 제거
quantile999 = seoul_bike_df['이용시간'].quantile(0.999)
seoul_bike_df = seoul_bike_df[(seoul_bike_df['이용시간'] < quantile999) & (seoul_bike_df['이용거리'] != 0) ]

# column명 변경
seoul_bike_df.rename(columns={'대여 대여소번호' : 'id' , '대여 대여소명':'주소2', '반납대여소번호' : 'id2'},inplace=True)

# 자전거 번호 정리
seoul_bike_df['자전거번호'] = seoul_bike_df['자전거번호'].str.replace('SPB-','').astype('int32')

# 대여일시 정리
rental_date = pd.to_datetime(seoul_bike_df['대여일시'])

# 대여 월 일, 대여 시간, 요일 추가
seoul_bike_df['month'] = rental_date.dt.month.astype('int8') #  int 8 크기  -128 ~ 127
seoul_bike_df['day'] = rental_date.dt.day.astype('int8')
seoul_bike_df['time'] = rental_date.dt.time
seoul_bike_df['weekday'] =rental_date.dt.weekday.astype('int8')

# type 변경
seoul_bike_df['id']  = seoul_bike_df['id'].astype('int16') #int 16 크기  -32768 ~ 32767
seoul_bike_df['id2']  = seoul_bike_df['id2'].astype('int16')
seoul_bike_df['이용시간']  = seoul_bike_df['이용시간'].astype('int16')

# 불필요한 column 제거
seoul_bike_df.drop(columns=['대여일시','대여거치대','반납일시','반납대여소명','반납거치대','주소2'],inplace=True)

# # long-lat 추가
# seoul_bike_df['id1_long'] = ''
# seoul_bike_df['id1_lat'] = ''
# seoul_bike_df['id2_long'] = ''
# seoul_bike_df['id2_lat'] = ''
# seoul_bike.columns=['month','day','time','weekday','by_id','st_id1','st_id2','riding_time','dist','id1_long','id1_lat','id2_long','id2_lat']
# seoul_bike = seoul_bike_df[['month','day','time','weekday','자전거번호','id','id2','이용시간','이용거리','id1_long','id1_lat','id2_long','id2_lat']]

# 



#위치 변경 및 column명 변경
seoul_bike = seoul_bike_df[['month','day','time','weekday','자전거번호','id','id2','이용시간','이용거리']]
seoul_bike.columns=['month','day','time','weekday','by_id','st_id1','st_id2','riding_time','dist']


#정비소 제거 
del_ids= np.array([  9992,   9998, -31073,   9980,      5,   9999,      3,   9997,  9990], dtype='int16')
seoul_bike = seoul_bike[~(seoul_bike['st_id2'].isin(del_ids)) | ~(seoul_bike['st_id1'].isin(del_ids))]

seoul_bike.reset_index(drop=True,inplace=True)


seoul_bike.head()

,month,day,time,weekday,by_id,st_id1,st_id2,riding_time,dist
0,1,13,21:02:05,2,50952,3,668,11,1853.99
1,1,18,18:02:08,0,52380,3,540,13,2291.05
2,1,20,18:01:19,2,53160,3,668,11,1508.98
3,1,22,17:46:34,4,4151,3,529,16,1220.00
4,1,22,17:54:26,4,30145,3,668,20,1991.86


In [32]:
seoul_bike.to_hdf('preprocessed_bike_2_2.hdf5',key='bike22')

d:\anaconda3\lib\site-packages\pandas\core\generic.py:2703: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->time,key->block1_values] [items->Index(['time'], dtype='object')]

  pytables.to_hdf(


불필요한 id 제거
<br>

반납대여소번호	반납대여소명
<br>
3	중랑센터
<br>
9992	사대문센터
<br>
9998	영남주차장 정비센터
<br>
9980	에이텍
<br>
5	상암센터 정비실
<br>
9999	상담센터
<br>
9997	강남센터
<br>
9990	_도봉센터

In [114]:
# 4개 주소 중복
# 이중 한양수자인아파트 앞 id 4811은 사용하지 않음
# 국회3문, 천왕이펜하우스5단지 앞은 완전 같음
# 신봉천주유소는 A,B로 구분
# 쓰다써..

seoul_bike["id"] : 2420개 대여소
seoul_bike["주소2"] : 2416개 대여소


한양수자인아파트 앞         2
국회3문               2
천왕이펜하우스5단지 앞       2
신봉천주유소             2
국립국악중,고교 정문 맞은편    1
                  ..
등촌2동주민센터           1
용문사 버스정류장          1
목동사거리 버스정류장        1
곰달래 문화복지센터 1-1     1
잠실나들목6             1
Length: 2416, dtype: int64

### 개별 값에 위경도 넣기(필요시)

In [ ]:
### st_id1에 맞는 위도 경도 넣기
for id in seoul_bike["st_id1"].unique() :
    seoul_bike.loc[(seoul_bike['st_id1']==id),'id2_long'] = bs_2109.query('id==@id')['위도'].values[0]
    seoul_bike.loc[(seoul_bike['st_id1']==id),'id2_lat'] = bs_2109.query('id==@id')['경도'].values[0]


### st_id2에 맞는 위도 경도 넣기
bike_station_df_2106 = pd.read_csv('D:/git_local_repository/data_for_project/seoul_bike/bike_station_2021.06.csv')

id_2106 = seoul_bike['st_id2'].isin(bike_station_df_2106['id'])

## bike_station_df_2106에 있는 위도 경도 st_id2에 넣기
for id in seoul_bike[id_2106]['st_id2'].unique() :
    seoul_bike.loc[(seoul_bike['st_id2']==id),'id2_long'] = bike_station_df_2106.query('id==@id')['위도'].values[0]
    seoul_bike.loc[(seoul_bike['st_id2']==id),'id2_lat'] = bike_station_df_2106.query('id==@id')['경도'].values[0]

## bike_station_2106에 없는 위도 경도 bike_station_df_2112에서 찾기
for id in seoul_bike[~id_2106]['st_id2'].unique() :
    seoul_bike.loc[(seoul_bike['st_id2']==id),'id2_long'] = bs_2109.query('id==@id')['위도'].values[0]
    seoul_bike.loc[(seoul_bike['st_id2']==id),'id2_lat'] = bs_2109.query('id==@id')['경도'].values[0]


In [74]:
# # ~21년 12월 대여소 정보
bike_station_df_2112 = pd.read_csv('D:/git_local_repository/data_for_project/seoul_bike/bike_station_2021.12.csv')
# # ~21년 6월 대여소 정보
bike_station_df_2106 = pd.read_csv('D:/git_local_repository/data_for_project/seoul_bike/bike_station_2021.06.csv')
# # ~21년 12월 대여소 정보
bike_station_df_2101 = pd.read_csv('D:/git_local_repository/data_for_project/seoul_bike/bike_station_2021.01.csv',encoding='CP949')



In [110]:
# 대여소 정보 합치기
a = pd.merge(bike_station_df_2112,bike_station_df_2106,how='outer',on=['대여소 번호','주소2','주소1'])
b = pd.merge(a, bike_station_df_2101, how='outer',on=['대여소 번호','주소2','주소1'])

,대여소 번호,주소2,지역_x,주소1,위도_x,경도_x,설치일_x,지역_y,위도_y,경도_y,설치일_y,지역,위도,경도,설치일
0,102,망원역 1번출구 앞,마포구,서울특별시 마포구 월드컵로 72,37.555649,126.910629,2015-09-06,마포구,37.555649,126.910629,2015-09-06,마포구,37.555649,126.910629,2015-09-06
1,103,망원역 2번출구 앞,마포구,서울특별시 마포구 월드컵로 79,37.554951,126.910835,2015-09-06,마포구,37.554951,126.910835,2015-09-06,마포구,37.554951,126.910835,2015-09-06
2,104,합정역 1번출구 앞,마포구,서울특별시 마포구 양화로 59,37.550629,126.914986,2015-09-06,마포구,37.550629,126.914986,2015-09-06,마포구,37.550629,126.914986,2015-09-06
3,105,합정역 5번출구 앞,마포구,서울특별시 마포구 양화로 48,37.550007,126.914825,2015-09-06,마포구,37.550007,126.914825,2015-09-06,마포구,37.550007,126.914825,2015-09-06
4,106,합정역 7번출구 앞,마포구,서울특별시 마포구 독막로 4,37.548645,126.912826,2015-09-06,마포구,37.548645,126.912826,2015-09-06,마포구,37.548645,126.912827,2015-09-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2651,3304,낙성대역 5번 출구,NaN,남부순환로 1917,NaN,NaN,NaN,NaN,NaN,NaN,NaN,관악구,37.477753,126.962364,2020-04-29
2652,2292,무지개아파트 앞,NaN,서울특별시 서초구 효령로 391,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서초구,37.488243,127.027016,2017-12-22
2653,2543,경문고교 사거리,NaN,서초구 동작대로 164,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서초구,37.491264,126.982956,2020-02-23
2654,2386,논현동 279-165,NaN,논현동 279-166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,강남구,37.509586,127.040909,2020-08-10


In [125]:
# 지역, 위도, 경도, 설치일 Nan 채우기

for name in ['지역','위도','경도','설치일'] :
    b.loc[b[f'{name}'].isna(),f'{name}'] = b.loc[b[f'{name}'].isna(),f'{name}_x'].values
    b.loc[b[f'{name}'].isna(),f'{name}'] = b.loc[b[f'{name}'].isna(),f'{name}_y'].values


,대여소 번호,주소2,지역_x,주소1,위도_x,경도_x,설치일_x,지역_y,위도_y,경도_y,설치일_y,지역,위도,경도,설치일
2638,3013,서울지방고용노동청 앞,중구,서울 중구 삼일대로 363,NaN,NaN,NaN,NaN,NaN,NaN,NaN,중구,NaN,NaN,2020-06-30
2646,4205,디지털미디어씨티역2번출구,마포구,마포구 상암동 1171-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,마포구,NaN,NaN,2020-12-02


In [132]:
# 저장
end = b[['대여소 번호', '주소2','주소1','지역','위도','경도','설치일']]

end.to_csv('D:\git_local_repository\data_for_project\seoul_bike\seoul_bike_station_01_12.csv',encoding='CP949')

,대여소 번호,주소2,주소1,지역,위도,경도,설치일
0,102,망원역 1번출구 앞,서울특별시 마포구 월드컵로 72,마포구,37.555649,126.910629,2015-09-06
1,103,망원역 2번출구 앞,서울특별시 마포구 월드컵로 79,마포구,37.554951,126.910835,2015-09-06
2,104,합정역 1번출구 앞,서울특별시 마포구 양화로 59,마포구,37.550629,126.914986,2015-09-06
3,105,합정역 5번출구 앞,서울특별시 마포구 양화로 48,마포구,37.550007,126.914825,2015-09-06
4,106,합정역 7번출구 앞,서울특별시 마포구 독막로 4,마포구,37.548645,126.912827,2015-09-06
...,...,...,...,...,...,...,...
2651,3304,낙성대역 5번 출구,남부순환로 1917,관악구,37.477753,126.962364,2020-04-29
2652,2292,무지개아파트 앞,서울특별시 서초구 효령로 391,서초구,37.488243,127.027016,2017-12-22
2653,2543,경문고교 사거리,서초구 동작대로 164,서초구,37.491264,126.982956,2020-02-23
2654,2386,논현동 279-165,논현동 279-166,강남구,37.509586,127.040909,2020-08-10


### 대여소 정리 안쓰는 코드 

In [ ]:
### 안쓰는 code
# # bike_station_df_2205.rename(columns={'대여소_ID' : 'id'},inplace=True)
# # bike_station_df_2205['id'] = bike_id = bike_station_df_2205['id'].str.replace('ST-','').astype('int16')
# # print(f'bike_station_22년 05월까지 : {len(bike_station_df_2205)}개 대여소')


# # 대여소 id 양식 통일
# bike_station_df_2112.rename(columns={'대여소 번호' : 'id'},inplace=True)
# seoul_bike_df.rename(columns={'대여 대여소번호' : 'id' , '대여 대여소명':'주소2', '반납대여소번호' : 'id2'},inplace=True)

# bike_station_df_2112['id'] = bike_station_df_2112['id'].astype('int16')

# from datetime import datetime
# BM = bike_station_df_2112['설치일'] < datetime(2021, 10, 1)


# print(f'bike_station 21년 9월 30일까지 대여소 : {len(bike_station_df_2112[BM])}개 대여소')
# print(f'seoul_bike_df["id"] : {len(seoul_bike_df["id"].unique())}개 대여소')
# print(f'seoul_bike_df["주소2"] : {len(seoul_bike_df["주소2"].unique())}개 대여소')

# # 21년 9월에 사용된 bike station 추출
# BM_2 = bike_station_df_2112['id'].isin(seoul_bike_df['id'])
# bs_2109=bike_station_df_2112[BM & BM_2]


# seoul_bike = seoul_bike_df[seoul_bike_df['id'].isin(bs_2109['id'])]
# seoul_bike.info()


# # id 2420개 주소2 2416개
# print(f'seoul_bike["id"] : {len(seoul_bike["id"].unique())}개 대여소')
# print(f'seoul_bike["주소2"] : {len(seoul_bike["주소2"].unique())}개 대여소')


# a = seoul_bike.groupby(['id','주소2']).size()
# b = [a.index[i][1] for i in range(len(a))]
# pd.Series(b).value_counts()